In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
import shutil 
import os
import gc
import time

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
save_path = r"/content"
destination = r"/content/drive/MyDrive/"

In [ ]:
urls = []
# add proper regex for link
regex = re.compile(r"https://www.youtube.com/watch\?v=.*")

In [ ]:
while True:
    url = input("Enter Url Or Type Exit: ")
    if url.lower() == "exit":
        break
    urls.append(url)

In [ ]:
print(f"{len(urls)} videos start to download")
downloaded = os.listdir(destination)

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    title = soup.find("title").text.split("-")
    filename = title[0]+" - "+title[1]
    filename = filename.strip()+".mp4"

    if filename in downloaded:
        source = f"{save_path}/{filename}"
        try:
            print(filename, "already downloaded")
            urls.remove(url)
            shutil.rmtree(source)
            gc.collect()
        except:
            continue

    script_tags = soup.find_all("script", type="text/javascript")
    video_url = ""

    for script_tag in script_tags:
        script_content = script_tag.text
        video_matches = regex.findall(script_content)
        if video_matches:
            video_url = video_matches[0]

    # send a GET request to the video URL to get the video data
    response = requests.get(video_url, stream=True)

    # get the total size of the video in bytes
    total_size = int(response.headers.get("Content-Length", 0))

    # create a progress bar using tqdm
    progress_bar = tqdm(total=total_size, unit="B", unit_scale=True)

    # write the video data to a file and update the progress bar
    with open(filename, "wb") as f:
        print(filename, " start download")
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
                # progress_bar.update(len(chunk))

    source = f"{save_path}/{filename}"
    print(source, destination)
    try:
        print("moves")
        shutil.move(source, destination)
        print("moved")
    except:
        continue

    progress_bar.close()
    urls.remove(url)
    print(filename, " downloaded successfully!")
    gc.collect()

    time.sleep(30)

print("All Downloaded Successfully!!!")